# multisizerdata1.0
### Script to read and analyze raw data from multisizer (.#m4 files) - by Marco Fumasoni

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt

# Ensure the necessary functions are defined

def parse_multisizer_file(file_path):
    def find_value_in_file(file_path, line_number, prefix):
        with open(file_path, 'r') as file:
            lines = file.readlines()
            
            if len(lines) < line_number:
                raise ValueError(f"The file does not have {line_number} lines.")
            
            line = lines[line_number - 1].strip()
            
            if line.startswith(prefix):
                value = line.split('=')[1].strip()
                return float(value)
            else:
                raise ValueError(f"The {line_number}th line does not contain '{prefix}'.")

    Kd = find_value_in_file(file_path, 127, "Kd=")
    nTimestamps = find_value_in_file(file_path, 136, "nTimeStamps=")
    Current = find_value_in_file(file_path, 178, "Current=") / 1000 # milliamperes
    Aperture = (find_value_in_file(file_path, 182, "Aperture=")) / 1000000 # micrometers
    Gain = find_value_in_file(file_path, 184, "Gain=")
    Resistance = 25*Gain # convert gain to equiv resistance (kohms)
    MaxHtCorr = find_value_in_file(file_path, 187, "MaxHtCorr=")
    Time = find_value_in_file(file_path, 199, "Time=")
    countspervolt = 1 / (4 * 298.02e-9) # defined constant
    
    data = []
    metadata = {}
    in_pulses_section = False

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line:
                continue

            if line.startswith('[#Pulses5hex]'):
                in_pulses_section = True
                continue
            elif line.startswith('[#TSms]') or line.startswith('[#TSpulses]'):
                in_pulses_section = False
                continue

            if in_pulses_section:
                values = line.split(',')
                if len(values) == 5:
                    event_data = {
                        'MaxHeight': int(values[0], 16),
                        'MidHeight': int(values[1], 16),
                        'Width': int(values[2], 16),
                        'Area': int(values[3], 16),
                        'Gain': int(values[4], 16),
                    }
                    data.append(event_data)
            else:
                if '=' in line:
                    key, value = line.split('=', 1)
                    metadata[key.strip()] = value.strip()

    df = pd.DataFrame(data, columns=['MaxHeight', 'MidHeight', 'Width', 'Area', 'Gain'])
    
    df['Height'] = df['MaxHeight'] + MaxHtCorr
    df['Diameter'] = Kd * ((df['Height'] / (countspervolt * Resistance * Current)) ** (1/3))
    pi = np.pi
    df['Volume'] = (4/3) * pi * (df['Diameter']/2)**3
    
    return df

def calculate_KDE(data, bw):
    kde = gaussian_kde(data, bw_method=bw)
    x = np.linspace(data.min(), data.max(), 1000)
    density = kde(x)
    area_under_curve = np.trapz(density, x)
    density /= area_under_curve
    kde_data = pd.DataFrame({'x': x, 'density': density})
    return kde_data

def filter_n_smooth(df, maxV, bw):
    df_filtered = df[df['Volume'] <= maxV]
    dD = calculate_KDE(df_filtered['Diameter'], bw)
    dV = calculate_KDE(df_filtered['Volume'], bw)
    return df_filtered, dD, dV

# Define directories and file patterns
directory = '/Users/mariananatalino/Desktop/Paper/Size/'
output_dir = os.path.join(directory, 'plots_histogram')
os.makedirs(output_dir, exist_ok=True)

# Initialize dictionaries to store dataframes
data = {}
diameter = {}
volume = {}

# List all files with the .m4 extension in the directory
file_list = glob.glob(os.path.join(directory, '*.#m4'))

maxV = 1000 # Set a different max volume if you are working with very big cells
bw = 0.05 # try changing this to refine the density distribution so that it matches the histogram

def extract_metadata(filename):
    parts = filename.split('_')
    genotype = 'wild type' if parts[1][0] == 'w' else 'ctf4Δ mutant'
    glucose_concentration = parts[1][1:] + '%'
    replica_number = parts[2]
    return genotype, glucose_concentration, replica_number

# Iterate through each file and process it
for file_path in file_list:
    # Parse the file
    df = parse_multisizer_file(file_path)
    
    # Filter and smooth the data
    df2, dD, dV = filter_n_smooth(df, maxV, bw)
    
    # Extract the filename without the directory and extension
    base_name = os.path.basename(file_path)
    filename_without_extension = base_name.split('.#')[0]
    
    # Extract metadata from filename
    genotype, glucose_concentration, replica_number = extract_metadata(filename_without_extension)
    
    # Store the dataframes in the respective dictionaries
    data[filename_without_extension] = df2
    diameter[filename_without_extension] = dD
    volume[filename_without_extension] = dV
    
    # Create a figure and two subplots arranged horizontally
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))

    # Volume plot
    # Plot histogram on the first subplot
    ax1.hist(df2['Volume'], bins=600, density=True, edgecolor='w', linewidth=0.5, alpha=0.2)
    ax1.plot(dV['x'], dV['density'], color='k', label='Density')

    # Calculate mean, median, and mode for volume
    mean_volume = np.average(dV['x'], weights=dV['density'])
    median_volume = np.interp(0.5, np.cumsum(dV['density']) / np.sum(dV['density']), dV['x'])
    mode_volume = dV['x'][np.argmax(dV['density'])]

    # Plot lines for mean, median, and mode for volume
    ax1.axvline(mean_volume, color='r', linestyle='--', label=f'Mean: {mean_volume:.2f}')
    ax1.axvline(median_volume, color='y', linestyle='--', label=f'Median: {median_volume:.2f}')
    ax1.axvline(mode_volume, color='b', linestyle='--', label=f'Mode: {mode_volume:.2f}')

    # Customize plot for volume
    ax1.set_xlim(0, 500)
    ax1.legend(frameon=False)
    ax1.set_title(f'Volume Distribution ({filename_without_extension})')
    ax1.set_xlabel('Volume (fL)')
    ax1.set_ylabel('Density')

    # Diameter plot
    # Plot histogram on the second subplot
    ax2.hist(df2['Diameter'], bins=100, density=True, edgecolor='w', linewidth=0.5, alpha=0.2)
    ax2.plot(dD['x'], dD['density'], color='k', label='Density')

    # Calculate mean, median, and mode for diameter
    mean_diameter = np.average(dD['x'], weights=dD['density'])
    median_diameter = np.interp(0.5, np.cumsum(dD['density']) / np.sum(dD['density']), dD['x'])
    mode_diameter = dD['x'][np.argmax(dD['density'])]

    # Plot lines for mean, median, and mode for diameter
    ax2.axvline(mean_diameter, color='r', linestyle='--', label=f'Mean: {mean_diameter:.2f}')
    ax2.axvline(median_diameter, color='y', linestyle='--', label=f'Median: {median_diameter:.2f}')
    ax2.axvline(mode_diameter, color='b', linestyle='--', label=f'Mode: {mode_diameter:.2f}')

    # Customize plot for diameter
    ax2.set_xlim(0, 20)
    ax2.legend(frameon=False)
    ax2.set_title(f'Diameter Distribution ({filename_without_extension})')
    ax2.set_xlabel('Diameter (µm)')
    ax2.set_ylabel('Density')

    # Adjust layout to prevent overlap
    plt.tight_layout()

    # Save the plot
    output_path = os.path.join(output_dir, f'{filename_without_extension}_histogram.png')
    plt.savefig(output_path)
    plt.close()

    print(f'Plot saved for: {filename_without_extension}')
